# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [85]:
# all import statements needed for the project, for example:

import math
import re
import os
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import geopandas
import requests
import sqlalchemy as db

In [2]:
# any general notebook setup, like log formatting

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [X] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [60]:
def calculate_distance(from_coord, to_coord): # from_coord, to_coord = [lat, lon]
    # radius of earth in km
    R = 6371

    lat_from = math.radians(from_coord[0])
    lon_from = math.radians(from_coord[1])
    lat_to = math.radians(to_coord[0])
    lon_to = math.radians(to_coord[1])

    # Haversine formula: https://en.wikipedia.org/wiki/Haversine_formula
    dlon = lon_to - lon_from
    dlat = lat_to - lat_from
    a = math.sin(dlat / 2)**2 + math.cos(lat_from) * math.cos(lat_to) * math.sin(dlon / 2)**2
    d = 2 * R * math.asin(math.sqrt(a))
    
    return d   
    

In [49]:
def add_distance_column(dataframe): 
    # dataframe contains pickup_latitude etc. after column name normalization   
    dataframe['distance'] = dataframe.apply(
        lambda x: calculate_distance([x["pickup_latitude"],x["pickup_longitude"]],
                                     [x["dropoff_latitude"],x["dropoff_longitude"]]),
        axis = 1
    )
    
    

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [87]:
import warnings
warnings.filterwarnings("ignore")

In [89]:
zones = geopandas.read_file("taxi_zones.shp")
zones = zones.to_crs(4326)
zones['lon'] = zones.centroid.x
zones['lat'] = zones.centroid.y

zones.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,lon,lat
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695...",-74.174000,40.691831
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",-73.831299,40.616745
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870...",-73.847422,40.864474
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725...",-73.976968,40.723752
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562...",-74.188484,40.552659


In [6]:
def get_taxi_html():
    response = requests.get(TAXI_URL)
    html = response.content
    return html

In [82]:
def find_taxi_csv_urls():
    pattern = r"Yellow Taxi Trip Records"
    content = get_taxi_html()
    soup = bs4.BeautifulSoup(content, 'html.parser')
    y_urls = [y['href'] for y in soup.find_all(title=pattern)]
    return y_urls

In [83]:
def get_coordinate(LocationID):
    Lon = zones[zones["LocationID"] == LocationID]['lon'].values[0]
    Lat = zones[zones["LocationID"] == LocationID]['lat'].values[0]
    
    return [Lat, Lon]

looking up the latitude and longitude for some months where only location IDs are given for pickups and dropoffs (use the shp file linked above) – some location IDs may not be valid so those specific trips can be removed; (updated Nov 3, 2022)  
removing unnecessary columns and only keeping columns needed to answer questions in the other parts of this project;
removing invalid data points (use your discretion!);  
normalizing column names;  
normalizing and using appropriate column types for the respective data;  
for Uber and Yellow Taxi data, removing trips that start and/or end outside of the following latitude/longitude coordinate box: (40.560445, -74.242330) and (40.908524, -73.717047).  

Sampling: Each month of Yellow Taxi data contains millions of trips. However, the provided Uber dataset is only a sampling of all data. Therefore, you will need to generate a sampling of Yellow Taxi data that’s roughly equal to the sample size of the Uber dataset. 



In [ ]:
def get_and_clean_month_taxi_data(url):
    # download taxi data with url and save as dataframe
    df = pd.read_parquet(url, engine='pyarrow')
    
    #sampling
    
    # looking up the latitude and longitude for some months where only location IDs
    pattern1 = r"LocationID"
    pattern2 = r"latitude"
    pattern3 = r"Lat"
    df_type = 0
    for col in df.columns:
        if re.search(pattern1, col):
            df_type = 1 # 2011-01 to 2015-06
        elif re.search(pattern2, col):
            df_type = 2 # 2010-01 to 2010-12
        else:
            df_type = 3 # 2009-01 to 2009-12
            
    if df_type == 1:
        # drop unnecessary columns
        cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime','passenger_count', 'trip_distance', 'PULocationID', 
        'DOLocationID', 'fare_amount','tip_amount', 'total_amount']
        df = df[cols]
        
        #
        df["pickup_latitude"] = df.apply(
            lambda x: get_coordinate(x["PULocationID"])
            axis = 1
        )
        
        
        # normalize column names and types
        df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)
    
    # seperate pickup date and time
   
    # sampling roughly the same amount of data 
    
    # remove trips out side the designated coordbox (40.560445, -74.242330) and (40.908524, -73.717047)

    # normalize column names and types
    
    # drop unnecessary columns
    
    # save to a csv file 
    
    return df

In [ ]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_url:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [102]:
def load_and_clean_uber_data(csv_file):
    file = pd.read_csv(csv_file)

    # drop OTC in datetime
    new = file["pickup_datetime"].str.split(" ", n = 2, expand = True)
    file["pickup_datetime"] = new[0] + " " + new[1]
    
    # convert to datetime
    file["pickup_datetime"] = pd.to_datetime(file["pickup_datetime"])  
     
    # remove trips out side the designated coordbox (40.560445, -74.242330) and (40.908524, -73.717047)
    file = file[(file["pickup_latitude"] >= 40.560445) & (file["pickup_latitude"] <= 40.908524)]
    file = file[(file["pickup_longitude"] >= -74.242330) & (file["pickup_longitude"] <= -73.717047)]
    file = file[(file["dropoff_latitude"] >= 40.560445) & (file["dropoff_latitude"] <= 40.908524)]
    file = file[(file["dropoff_longitude"] >= -74.242330) & (file["dropoff_longitude"] <= -73.717047)]
    
    # drop 0 passenger trips
    file = file[file["passenger_count"] != 0]
    
    # drop trips with no fare 
    file = file[file["fare_amount"] != 0]
    
    # drop trips with no distance between dropoff and pickup
    file = file[(file["pickup_latitude"] != file["dropoff_latitude"]) 
                & (file["pickup_longitude"] != file["dropoff_longitude"])]
   
    # drop unnecessary columns
    file.drop(['Unnamed: 0', 'key'], axis=1, inplace=True)
    
    return file

In [91]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    
    return uber_dataframe

In [103]:
uber_dataframe = load_and_clean_uber_data(UBER_CSV)
uber_data = get_uber_data()
uber_data.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,7.5,2015-05-07 19:52:06,-73.999817,40.738354,-73.999512,40.723217,1,1.683323
1,7.7,2009-07-17 20:04:56,-73.994355,40.728225,-73.994710,40.750325,1,2.457590
2,12.9,2009-08-24 21:45:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377
3,5.3,2009-06-26 08:22:21,-73.976124,40.790844,-73.965316,40.803349,3,1.661683
4,16.0,2014-08-28 17:47:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450


In [104]:
uber_data.shape

(192754, 8)

In [72]:
# check nan values
uber_data.isna().sum()

fare_amount          0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
pickup_date          0
pickup_time          0
distance             0
dtype: int64

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [11]:
def clean_month_weather_data_hourly(csv_file):
    const_col = ["DATE","LATITUDE","LONGITUDE"]
    hourly_cols = ["HourlyPrecipitation", "HourlyWindSpeed"]
    daily_col = ["DailyPrecipitation", "DailyAverageWindSpeed", "DailySustainedWindSpeed"]
    # drop nan values
    
    # calculate missing hourly data with daily data

In [ ]:
def clean_month_weather_data_daily(csv_file):
    raise NotImplemented()

In [ ]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["TODO"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [14]:
engine = db.create_engine(DATABASE_URL)

In [15]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [16]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [17]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [18]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [19]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)